# Resumen

En esta práctica, ilustraremos conceptos básicos sobre el modelo booleano de recuperación de información vistos en las clases de teoría sobre un corpus de documentos real. Implementaremos dicho modelo de RI tanto mediante una matriz término-documento, como mediante índices invertidos. Aprenderemos a construir índices invertidos mediante enfoques sencillos con el fin de ilustrar su funcionamiento, pero sin atender a las limitaciones impuestas por el hardware subyacente. En concreto, las tareas a realizar en esta práctica son:

1. **Familiarizarse con un *corpus* de documentos** de texto plano generados a partir de sitios web de la Universidad de Stanford.
1. **Recordar la técnica de *grepping***, como forma básica de escanear los documentos del corpus en busca de aquellos que contienen los términos de una consulta.
1. **Construir una matriz término-documento**  para un subconjunto de documentos y realizaremos algunas consultas booleanas a partir de la misma. 
1. **Construir un índice invertido en memoria**, sin considerar restricciones de ningún tipo impuestas por el hardware a la hora de su construcción.
1. **Recuperar información a partir del índice**, mediante el procesamiento de consultas booleanas conjuntivas y disyuntivas utilizando el índice invertido creado.
1. **Optimización de consultas**, evaluando el rendimiento de los diferentes planes de ejecución e incluyendo el uso de *skip pointers* para acelerar el proceso.

# *Corpus* de documentos

El corpus con el que trabajaremos en esta tarea consta de aproximadamente 100K documentos, que están disponibles como archivo .zip en: http://ditec.um.es/~lfmaimo/docencia/ri/practica1.zip, junto con algunos otros ficheros de prueba. El siguiente código descarga el material en el directorio actual.

In [1]:
# Añade aquí los imports que necesites
import sys
import os
import numpy as np
from pathlib import Path

In [2]:
from urllib.parse import urlparse
import urllib.request
import zipfile
# URL del corpus de documentos
# OJO, solo se descargará si no existe ya el fichero zip
data_url = 'http://ditec.um.es/~lfmaimo/docencia/ri/practica1-datos.zip'

local_filename = os.path.basename(urlparse(data_url).path)
if not Path(local_filename).exists():
    print(f'Descargando {data_url} a {local_filename}...')
    urllib.request.urlretrieve(data_url, local_filename)
zip_ref = zipfile.ZipFile(local_filename, 'r')
zip_ref.extractall()
zip_ref.close()

Descargando http://ditec.um.es/~lfmaimo/docencia/ri/practica1-datos.zip a practica1-datos.zip...


In [3]:
corpus_dir='data/corpus'
toy_dir = 'data/toy'
if not Path(corpus_dir).is_dir():
    raise FileNotFoundError(f"No puedo encontrar el directorio '{path}'.")
if not Path(toy_dir).is_dir():
    raise FileNotFoundError(f"No puedo encontrar el directorio '{path}'.")

El tamaño del corpus de documentos a indexar es de unos 420 MB.
El corpus consiste en documentos de texto plano generados a partir de páginas web, en las que se ha eliminado toda la información HTML.

In [4]:
!du -hs $corpus_dir

420M	data/corpus


Hay 10 subdirectorios (denominados 0-9) bajo el directorio de datos que contiene el corpus. Cada directorio contiene aproximadamente 10000 ficheros.

In [5]:
print(f"Contents of {corpus_dir}: {sorted(os.listdir(corpus_dir))}")

for subdir in sorted(os.listdir(corpus_dir)):
    print(f"Directory '{subdir}' has {len(os.listdir(Path(corpus_dir)/subdir))} files")

Contents of data/corpus: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Directory '0' has 10000 files
Directory '1' has 10000 files
Directory '2' has 10000 files
Directory '3' has 10000 files
Directory '4' has 9999 files
Directory '5' has 9997 files
Directory '6' has 9998 files
Directory '7' has 10000 files
Directory '8' has 10000 files
Directory '9' has 9004 files


Cada archivo de cada subdirectorio es el contenido de una página web individual. Los nombres de los archivos individuales son únicos dentro de cada subdirectorio, pero no son necesariamente únicos entre subdirectorios distintos (es decir, la ruta completa de los archivos es única).
La siguiente celda muestra los primeros 10 ficheros del subdirectorio '0' del corpus de documentos.

In [6]:
sorted(os.listdir(Path(corpus_dir)/'0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

El corpus de documentos ya ha sido *tokenizado*, de forma que cada documento sólo contiene palabras delimitadas por espacios. La siguiente celda muestra el contenido de un fichero cualquiera del corpus:

In [7]:
with open(Path(corpus_dir)/'0/crowds.stanford.edu_main.html', 'r') as f:
    print(f.read())

stanford humanities laboratory crowds crowd theorists semantic histories testimonies basement contact galleries stage sponsored by the stanford humanities laboratory and the seaver institute crowds is a collaborative research project which focuses on the rise and fall of the crowd particularly the revolutionary crowd in the western sociopolitical imagination between 1789 and the present crowd theorists semantic histories testimonies basement contact galleries revolutionary tides crowds intermedia loop1



También usaremos un pequeño corpus de documentos unos pocos KBs en la carpeta indicada por la variable `toy_dir`. Utilizaremos este conjunto de datos de juguete para probar nuestro código antes de ejecutarlo en el conjunto de datos completo. El índice de juguete generado se colocará en el directorio `out_toy_dir`:

In [8]:
!du -hs $toy_dir

40K	data/toy


## Vocabulario

En primer lugar, vamos a analizar el vocabulario del corpus de documentos. Empezaremos por leer el corpus de documentos, obtener el tamaño del vocabulario y calcular la frecuencia de cada palabra.

In [9]:
def read_files(corpus_dir):
    """Lee todos los ficheros de texto de los directorios y devuelve el contenido combinado."""
    total_content = ""
    
    for directory in os.listdir(corpus_dir):
        for filename in os.listdir(Path(corpus_dir)/directory):
            filepath = os.path.join(corpus_dir,directory, filename)
            if os.path.isfile(filepath):
                with open(filepath, 'r', encoding='utf-8') as file:
                    total_content += file.read()
    
    return total_content

Usaremos la clase `Counter` para programar una función que calcule las frecuencias de las palabras en el corpus de documentos así como el tamaño del vocabulario (`vocabulary_size`) y el número de palabras total en el corpus. Después, mostraremos las 10 palabras más frecuentes del corpus junto con el número de ocurrencias de cada una.

In [10]:
from collections import Counter

def calculate_frequencies(words):
    """Calcula la frecuencia de aparición de cada término en una lista de términos."""
    return Counter(words)

In [11]:
# Lee el corpus
text = read_files(corpus_dir)

# Extrae los términos de los documentos
words = text.split()
print(f"Número de términos de la colección: {len(words)}")

# Calcula frecuencias
frequencies = calculate_frequencies(words)
print(f"{frequencies.total()}")

# Muestra el tamaño del vocabulario
vocabulary_size = len(frequencies)
print(f"Vocabulary size: {vocabulary_size}")

# Imrprime las 10 palabras más frecuentes
most_common_words = frequencies.most_common(10)
print("The 10 most frequent words are:")
for word, freq in most_common_words:
    print(f"  {word}: {freq}")


Número de términos de la colección: 25498340
25498340
Vocabulary size: 347071
The 10 most frequent words are:
  the: 925694
  of: 596269
  and: 560904
  to: 432042
  stanford: 357767
  in: 336003
  a: 308648
  for: 257443
  on: 158702
  is: 152447


Resultado:
```
Number of terms in collection: 25498340
25498340
Vocabulary size: 347071
The 10 most frequent words are:
  the: 925694
  of: 596269
  and: 560904
  to: 432042
  stanford: 357767
  in: 336003
  a: 308648
  for: 257443
  on: 158702
  is: 152447
```

# Recuperación de información mediante *grepping*

Para pequeñas cantidades de datos de texto, podemos simplemente realizar un escaneo lineal a través de una colección de documentos para encontrar información específica utilizando expresiones regulares. Esto se conoce habitualmente como *grepping*, por el comando UNIX `grep`. Utilizado con las opciones adecuadas (puedes consultarlas con `man grep`), `grep` se puede usar para realizar recuperación de información sobre un corpus de documentos pequeño, realizando consultas más o menos complejas. Entre las opciones de `grep` más relevantes están: 
* `-r`: busca recursivamente en las rutas indicadas.
* `-l`: muestra el nombre del fichero coincidente en lugar de las líneas coincidentes
* `-w`: considera exclusivamente coincidencias en *palabras* completas (usando delimitadores habituales), excluyendo coincidencias en subcadenas.

### Ejemplo 1: documentos que contienen la palabra "eyeball"

In [12]:
!grep -rlw eyeball $corpus_dir

data/corpus/6/transportation.stanford.edu_pdf_five-simple-checks.pdf
data/corpus/8/www.stanford.edu_class_humbio103_ParaSites2006_Loiasis_Clinical%2520Presentation.html


### Ejemplo 2: documentos que contienen las palabras "murcia" o "alicante"

Una forma habitual de usar *grep* es con la opción `-E` que permite expresiones regulares extendidas capaces, entre otras cosas, de usar el operador lógico `|` para buscar simultáneamente más de un patrón de búsqueda.

Así, el siguiente comando encuentra todos los documentos que contienen las palabras "murcia" o "alicante".

In [13]:
!grep -Erlw 'alicante|murcia' $corpus_dir  | sort

data/corpus/0/art.stanford.edu_profile_Pamela%2bLee_
data/corpus/4/opera.stanford.edu_librettists_Piave.html
data/corpus/5/radiology.stanford.edu_patient_clinical_sections_
data/corpus/6/wais.stanford.edu_Spain_spain_andthesiesta7402.html


### Ejemplo 3: documentos que contienen las palabras "murcia" y "radiology"

Si queremos realizar una consulta conjuntiva (encontrar documentos que deban incluir todos los términos de la consulta), una forma de hacerlo es mediante el mecanismo de sustitución de comandos del *shell* (*command substitution*), el cual permite que la salida de un comando dado entre `$()` reemplace al propio comando antes de ejecutar el siguiente comando más externo. Así, un ejemplo (de dudosa utilidad) pero que ilustra *command substitution* sería: `ls $(echo "hola")`. Este comando es equivalente `ls hola`, pero utiliza *command substitution* de forma que el *shell* ejecutaría primero el comando entre `$()` (en este caso, `echo "hola"`) y la salida de dicho comando (en este caso, la cadena "hola") sustituiría al propio comando `$(echo "hola")` a la hora de ejecutar `ls`. Mediante este mecanismo es posible usar `grep` para obtener las rutas a los documentos que deban contener varios término de búsqueda ("and"). También se podría usar `xargs` combinado con tuberías.

Usando este método, podemos encontrar los documentos que contienen "murcia", y a partir de los ficheros encontrados, busca la palabra "radiology", de esta forma:

In [14]:
!grep -wl 'radiology' $(grep -rwl 'murcia' $corpus_dir)

data/corpus/5/radiology.stanford.edu_patient_clinical_sections_


## Ejercicio

In [25]:
# Obtén todos los documentos que contienen las palabras 'radiology', 'nuclear' y 'murcia'
!grep -wl "radiology" $(grep -rwl "nuclear" $(grep -rwl "murcia" $corpus_dir))

data/corpus/5/radiology.stanford.edu_patient_clinical_sections_


Sin embargo, el escaneo lineal de todos los documentos en busca del término consultado es una solución de *fuerza bruta* muy ineficiente. Aunque gracias a tecnologías de almacenameniento rápidas como los discos de estado sólido puede resultar factible en términos de latencia para un corpus de documentos relativamente pequeño, como el que utilizamos en esta práctica, el problema se agrava cuando el sistema debe procesar un mayor número de consultas simultáneas. En ese escenario, ni siquiera un disco rápido sería capaz de responder con los documentos relevantes a cada consulta planteada en un tiempo razonable. En general, escanear todos los documentos para realizar una consulta resulta muy lento, particularmente cuando el corpus es muy grande o hay que procesar un gran número de consultas. 

# Matriz término-documento

Para recuperar información sobre grandes cantidades de datos, necesitamos formas de hacer nuestras búsquedas más eficientes que el *grepping*, como son los índices invertidos. Sin embargo, antes de pasar a construir un índice, veremos con un enfoque inicial que también permite satisfacer la capacidad de realizar búsquedas booleanas: la matriz documento-término. Se trata de una forma de representar (muy dispersamente) la aparición de términos en una colección de documentos. A partir de dicha matriz de incidencia podemos construir un modelo booleano de recuperación.

Vamos a mostrar la matriz de incidencia término-documento de un subconjunto pequeño de documentos del corpus. Lo haremos sobre un número pequeño para poder visualizarla e interpretarla más fácilmente.

Para ello, en primer lugar debes programar una función que se encargue de leer los ficheros ubicados en un directorio dado como parámetro, y retornar una lista de tuplas, en la que cada tupla contendrá el contenido de cada fichero y su ruta relativa.

La función debe aceptar un segundo parámetro `scan_ratio` mediante el que, opcionalmente, se podrá indicar un *ratio* de ficheros a escanear menor que 1, de forma que se limite la lectura a los N primeros ficheros, en orden alfabético (*N = len(listdir) $\times$ scan_ratio*).

In [16]:
def read_files_in_directory(directory_path, scan_ratio=1):
    """Lee los ficheros de un directorio y devuelve una lista de pares (contenido, ruta).
        Args:
            directory_path (str): Ruta del directorio a escanear.
            scan_ratio (float): Proporción de ficheros a escanear (entre 0 y 1).
        Returns:
            list: Lista de tuplas (contenido, ruta) con los ficheros ordenados por nombre.
    """
    
    files = []
    filelist = os.listdir(directory_path)
    num_files = int(len(filelist)*scan_ratio)
    print(f"Escaneando {num_files} ficheros en '{directory_path}'")
    
    try:
        # Loop through the files in the directory
        for filename in sorted(filelist)[0:num_files]:
            filepath = os.path.join(directory_path, filename)
            # Check if it's a file
            if os.path.isfile(filepath):
                with open(filepath, 'r', encoding='utf-8') as file:
                    content = file.read()  # Read the entire file's content
                    # Add tuple with content and path to list
                    files.append((content,filepath))
    except FileNotFoundError:
        print(f"El directorio {directory_path} no existe.")
    except PermissionError:
        print(f"No tengo permiso para acceder a {directory_path}.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    
    return files

Prueba tu función escaneando el 0.2% (ratio de 0.002) de los ficheros ubicados en el subdirectorio `0` del corpus (los 20 primeros documentos).

In [17]:
# Para el ejemplo nos centraremos en el subdirectorio '0' 
directory_path = Path(corpus_dir)/'0'
# Cambia el scan_ratio para escanear más o menos ficheros...
scan_ratio = 0.002  # solo el 0.2% de los ficheros del directorio
# Obtén el nombre y contenido de los documentos
sample_docs = read_files_in_directory(directory_path, scan_ratio)
len(sample_docs)

Escaneando 20 ficheros en 'data/corpus/0'


20

Ejercicio: recopila el vocabulario de los 20 documentos escaneados anteriormente ( conjunto de todos los términos únicos que aparecen en dichos documentos). Deberías obtener como resultado una lista de 1929 términos distintos.

In [20]:
# Construye el  conjunto de todos los términos únicos
sample_unique_terms = {}
### Begin your code
# Recorre cada documento en sample_docs
for content, path in sample_docs:
    # Divide el contenido en palabras y las añade al conjunto
    words = content.split()
    for word in words:
        sample_unique_terms[word] = True

# Convierte el conjunto a lista para facilitar su uso
sample_unique_terms = list(sample_unique_terms.keys())
print(f"Número de términos únicos: {len(sample_unique_terms)}")
### End your code

Número de términos únicos: 1929


Ahora, programa una función `build_term_document_matrix` que reciba como argumentos la lista de tuplas obtenida anteriormente y el vocabulario, y construya una matriz término-documento utilizando un diccionario de listas para facilitar su interpretación (un diccionario de términos cuyo valor asociado es una lista de indicencia de N valores binarios, en el que el valor i-ésimo vale 1 si el término aparece en el documento i-ésimo de la colección, 0 en caso contrario.

In [21]:
# Construct a term-document matrix
# here as a Python dictionary for ease of interpretability
def build_term_document_matrix(docs, terms):
    doc_term_matrix = {}
    ### Begin your code

    # Inicializa la matriz con ceros por todos los términos
    for term in terms:
        doc_term_matrix[term] = [0] * len(docs)
    
    # Procesa cada documento
    for doc_idx, (content, path) in enumerate(docs):
        # Obtiene las palabras del documento
        words = content.split()
        # Convierte a conjunto para eliminar duplicados
        unique_words = set(words)

        # Marca presencia de cada palabra en este documento
        for word in words:
            if word in doc_term_matrix:
                doc_term_matrix[word][doc_idx] = 1
                
    ### End your code
    return doc_term_matrix

td_matrix = build_term_document_matrix(sample_docs, sample_unique_terms)
td_matrix

{'3d': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'radiology': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'lab': [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 'stanford': [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'university': [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'school': [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
 'of': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'medicine': [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'and': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'quantitative': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'imaging': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'in': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 'the': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'department': [1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,

In [24]:
td_matrix['jobs']

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Resultado:
```
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
```

Usando la matriz, podemos obtener fácilmente un array de NumPy que contiene vector de incidencia de un término 
cualquiera, y realizar consultas booleanas a partir de dichos vectores. Por ejemplo, usando operaciones booleanas con arrays de NumPy obtenidos a partir de la matriz término-documento, obtén los documentos que contienen las palabras "software", "idea" en las variables `v1` y `v2` respectivamente, y los que contienen ambos términos:

In [26]:
### Begin your code

# Obtener los vectores de incidencia para "software" e "idea"
v1 = np.array(td_matrix["software"])
v2 = np.array(td_matrix["idea"])

# Mostrar los vectores individuales
print(f"software:       {v1}")
print(f"idea:           {v2}")
print("----------------")

# Realizar la operación AND (intersección) usando &
print(f"software & idea:{v1 & v2}")
### End your code

software:       [1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
idea:           [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
----------------
software & idea:[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]


Resultado:

```
software:       [1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
idea:           [0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0]
----------------
software & idea:[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
```

Para obtener el nombre del fichero coincidente con la consulta, vamos a definir una función que dada una lista y un array de NumPy, filtre la lista quedándose únicamente con los que corresponden a los 1s del array.

In [27]:
def filter_by_array(input_list, binary_array):
    ### Begin your code
    # Comprueba que input_list y binary_array tienen la misma longitud
    assert len(input_list) == len(binary_array), "La lista y el array deben tener la misma longitud"

    # Usa una list comprehension para filtrar los elementos donde el array binario tiene un 1
    filtered_list = [input_list[i] for i in range(len(input_list)) if binary_array[i] == 1]

    ### End your code
    
    return filtered_list

Usando dicha función, podemos obtener una lista de rutas a ficheros coincidentes con la consulsta `"software" AND "idea"`. Usaremos `zip` para quedarnos con la lista de nombres de los ficheros del corpus y posteriormente filtrar dicha lista:

In [28]:
sample_file_contents, sample_file_paths = zip(*sample_docs)
sample_file_paths
filter_by_array(sample_file_paths, v1 & v2)

['data/corpus/0/aa.stanford.edu_']

Resultado:
```
['data/corpus/0/aa.stanford.edu_']
```

Ahora vamos comprobar si el resultado es correcto. Para ello, convertimos la lista de rutas a los ficheros a una cadena sin corchetes ni comas, que podamos usar para invocar a `grep` utilizando *command substitution*.

In [29]:
# Get the list of filenames in a suitable format for testing with grep
sample_file_paths_string = ' '.join(map(str, sample_file_paths))

In [30]:
!grep -wl 'ideas' $(grep -wl 'software' $sample_file_paths_string)

data/corpus/0/aa.stanford.edu_


Finalmente, comprobamos que las palabras "idea" y "software" aparecen en los documentos coincidentes:

In [31]:
docs_array = np.array(sample_file_contents, dtype='object')
result = [doc for doc in (v1 & v2) * docs_array if doc]
[("ideas" in text) & ("software" in text) for text in result ]

[True]

Ahora, realiza una consulta disyuntiva (*or*), para obtener los documentos que contienen las palabras "eye" o "feedback".

In [33]:
### Begin your code
# Obtener los vectores de incidencia para "eye" e "feedback"
v1 = np.array(td_matrix["eye"])
v2 = np.array(td_matrix["feedback"])

# Mostrar los vectores individuales
print(f"eye:            {v1}")
print(f"feedback:       {v2}")
print("----------------")

# Realizar la operación AND (intersección) usando &
print(f"eye | feedback: {v1 | v2}")
### End your code


eye:            [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
feedback:       [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
----------------
eye | feedback: [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


Resultado:

```
eye:           [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
feedback:      [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
---------------
eye | feedback:[0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
```

Al igual que antes, muestra únicamente los nombres de ficheros coincidentes:

In [35]:
### Begin your code
coincident_files = filter_by_array(sample_file_paths, v1 | v2)
for file in coincident_files:
    print(file)
### End your code

data/corpus/0/5-sure.stanford.edu_
data/corpus/0/aa.stanford.edu_about_control.php


Y por último, comprobamos mediante `grepping`.

In [36]:
!egrep -wl 'eye|feedback' $sample_file_paths_string

data/corpus/0/5-sure.stanford.edu_
data/corpus/0/aa.stanford.edu_about_control.php


## Tamaño de la matriz término-documento

Veamos cuál sería el tamaño en memoria de la matriz término-documento para el corpus completo que estamos considerando. En primer lugar, utiliza la función `read_files_in_directory` para escanear todos los ficheros de todos los directorios del corpus. Deberías obtener una lista de 98998 tuplas *(texto, ruta)*.

In [37]:
all_docs = []
# Change this to scan more or less files...
scan_ratio = 1 # 100% of collection

for subdir in sorted(os.listdir(corpus_dir)):
    directory_path = Path(corpus_dir)/subdir
    docs = read_files_in_directory(directory_path)
    all_docs.extend(docs)

print(f"Total files scanned: '{len(all_docs)}'")

Escaneando 10000 ficheros en 'data/corpus/0'
Escaneando 10000 ficheros en 'data/corpus/1'
Escaneando 10000 ficheros en 'data/corpus/2'
Escaneando 10000 ficheros en 'data/corpus/3'
Escaneando 9999 ficheros en 'data/corpus/4'
Escaneando 9997 ficheros en 'data/corpus/5'
Escaneando 9998 ficheros en 'data/corpus/6'
Escaneando 10000 ficheros en 'data/corpus/7'
Escaneando 10000 ficheros en 'data/corpus/8'
Escaneando 9004 ficheros en 'data/corpus/9'
Total files scanned: '98998'


Teniendo en cuenta el tamaño del vocabulario que calculaste anteriormente (`vocabulary_size`), la matriz término-documento de este corpus tendría 347071x98998 = 34359334858 elementos (34.4 giga-elementos)... por lo que no cabe en la memoria de la mayoría de computadores de sobremesa actuales.

In [38]:
from humanize import naturalsize

rows = len(frequencies)
cols = len(all_docs)
print(f"The term-document matrix has {rows}x{cols} = {naturalsize(rows*cols, binary=False)} elements")

The term-document matrix has 347071x98998 = 34.4 GB elements


Como podemos imaginar, si intentamos construir una matriz de tales dimensiones (por ejemplo, con NumPy), es muy posible, dependiendo de en qué hardware estemos ejecutando este notebook, que obtengamos un error, ya que no hay memoria suficiente para mantener dicha matriz en memoria. Y eso, a pesar de que tenemos un corpus de documentos relativamente pequeño (~100K documentos).

In [39]:
# Quizás una máquina potente pueda reservar una matriz de booleanos de unos 32GiB
a=np.zeros((rows,cols),dtype=bool)
bytes = sys.getsizeof(a)
print(f"The term-document matrix for this corpus takes {naturalsize(bytes, binary=True)} bytes")

# Difícilmente podría hacerse con enteros (256GiB)
#np.zeros((rows,cols),dtype=int)

MemoryError: Unable to allocate 32.0 GiB for an array with shape (347071, 98998) and data type bool

Veamos que en efecto la mayoría de los elementos de la matriz son ceros. Escribe una función que cuente el número de ceros que hay en una matriz término-documento dada en forma de diccionario de listas, como la que hemos construido antes. Luego, utiliza dicha función para calcular el número de elementos y el ratio de dispersión de la matriz creada a partir anteriormente a partir de un subconjunto de 20 documentos del corpus:

In [43]:
def count_zeros_and_elements(dict_of_lists):
    total_zeros = 0
    total_elements = 0
    ### Begin your code
    
    # Recorre cada término en el diccionario
    for term, doc_vector in dict_of_lists.items():
        # Para cada vector de documentos, cuenta los elementos totales
        total_elements += len(doc_vector)
        # Cuenta los ceros en este vector
        total_zeros += doc_vector.count(0)
    
    ### End your code
    return total_zeros, total_elements

zeros, elements = count_zeros_and_elements(td_matrix)
assert elements == len(sample_unique_terms) * len(sample_docs)
print(f"The sample term-document matrix has {len(sample_unique_terms)} x {len(sample_docs)} = {elements} elements.")
print(f"Its sparsity ratio is {1 - (elements-zeros)/elements}")

The sample term-document matrix has 1929 x 20 = 38580 elements.
Its sparsity ratio is 0.8959564541213064


Resultado:
```
The sample term-document matrix has 1929 x 20 = 38580 elements.
Its sparsity ratio is 0.8467599792638673
```

# Construcción de un índice invertido *naïve* en memoria

Un índice invertido asocia una colección de términos (vocabulario) con los documentos que contienen dichos términos. La estructura de datos es mucho más densa que una matriz de términos de documentos, puesto que sólo almacena las ocurrencias (los 1s de la matriz). 

Partiremos de la lista de tuplas *(contenido, ruta)* en la que previamente hemos leído todos los documentos del corpus junto con su ruta. Tomando dicha lista como argumento, vamos a programar una función `build_simple_inverted_index` que construya un índice en forma de diccionario de listas. Las claves del diccionario serán los términos, y para cada término tendremos una lista de ocurrencias asociada, con los números de documentos en los que aparece el término. La función debe recorrer la lista de documentos, asociando a cada uno de ellos un número de secuencia. Cada uno de los términos que contiene el documento (palabras obtenidas simplemente dividiendo la cadena con `split`) se añade al diccinario (si no lo está ya) y se asocia con el número de serie del documento actualmente procesado (si dicho *docID* no está ya en la *postings list*).

In [ ]:
# Construye un índice invertido
# usando un diccinario de Python por comodidad.
def build_simple_inverted_index(docs):
    index = {}
    ### Begin your code
    
    # Recorre cada documento asignándole un docID
    for doc_id, content in enumerate(docs):
        # Divide el contenido en palabras
        words = content.split()

        # Procesa cada palabra del documento
        for word in words:
            # Si el término no está en el índice, lo incluye
            if word not in index:
                index[word] = []

            # Si el docID no está ya en la posting list del término, lo añade
            if doc_id not in index[word]:
                index[word].append(doc_id)
    
    ### End your code
    return index

file_contents, file_paths = zip(*all_docs)
inverted_index = build_simple_inverted_index(file_contents)

Ahora podemos obtener la lista de ocurrencias (*postings lists*) para cualquier término del vocabulario, simplemente indicando el término como clave:

In [ ]:
posting_list = inverted_index['eyeball']
posting_list

La lista de ocurrencias contiene los *docIds* de los documentos en los que aparece el término, que en nuestro caso no es sino la posición del documento en la lista de tuplas `all_docs` sobre la que hemos construido el índice anteriormente.

Define una función `retrieve_docnames` que reciba una lista de rutas y una lista de docIDs, y devuelva los elementos de la lista de rutas a ficheros indicados por la lista de *docIds*. Luego, úsala para obtener los nombres de los documentos asociados a los *docIds* coincidentes con la consulta, haciendo uso de la lista de rutas almacenada en `file_paths`.

In [ ]:
def retrieve_docnames(filenames, docids):
  
    return [filenames[i] for i in list(docids)]
   
retrieve_docnames(file_paths,posting_list)

Como antes, comprueba que el resultado es correcto mediante *grepping*.

In [ ]:
!grep -rlw eyeball $corpus_dir

Resultado:

```
data/corpus/6/transportation.stanford.edu_pdf_five-simple-checks.pdf
data/corpus/8/www.stanford.edu_class_humbio103_ParaSites2006_Loiasis_Clinical%2520Presentation.html
```

# Procesamiento de consultas mediante un índice invertido

## Consultas disyuntivas (OR)

Programa una función que se encargue de calcular la unión de dos *postings lists* dadas como argumento, las cuales están ordenadas por *docID*. El resultado debe ser una lista con los *docIds* que contienen al menos uno de los dos términos, igualmente ordenada por *docId*.

In [ ]:
def or_postings(posting1, posting2):
    result = list()
    
    ### Begin your code
    i, j = 0, 0

    # Recorre ambas listas simultáneamente
    while i < len(posting1) and j < len(posting2):
        if posting1[i] == posting2[j]:
            # Si los docIDs son iguales, añade uno solo y avanza ambos punteros
            result.append(posting1[i])
            i += 1
            j += 1
        elif posting1[i] < posting2[j]:
            # Si el docID de posting1 es menor, lo añade y avanza puntero de posting1
            result.append(posting1[i])
            i += 1
        else:
            # Si el docID de posting2 es menor, lo añade y avanza puntero de posting2
            result.append(posting2[j])
            j += 1

        # Añade los elementos restantes de posting1 (si los hay)
        while i < len(posting1):
            result.append(posting1[i])
            i += 1

        # Añade los elementos restantes de posting2 (si los hay)
        while j < len(posting2):
            result.append(posting2[j])
            j += 1
    ### End your code
        
    return result

Ahora, utiliza la función anterior para procesar la consulta `"eyeball" OR "piercing"`, partiendo de las *postings lists* de cada uno de los términos obtenidas del índice.

In [ ]:
### Begin your code    
or_posting_list = ...


### End your code    

Resultado:

```
[18222, 40495, 48261, 64689, 65364, 65384, 65563, 68878, 85752]
```

In [ ]:
retrieve_docnames(file_paths,or_posting_list)

Por último, comprobamos si el resultado anterior es correcto mediante *grepping*. Usa una tubería para ordenar el resultado y obtener siempre la misma salida. Con la opción `-E` de `grep` (o,  equivalentemente, `egrep`) podemos buscar patrones extendidos utilizando expresiones regulares: por ejemplo, usar `|` en el patrón para buscar más de una palabra al mismo tiempo en un grupo de documentos.

In [ ]:
!egrep -rlw 'eyeball|piercing' $corpus_dir  | sort

Resultado:

```
data/corpus/1/events.stanford.edu_events_288_28851_
data/corpus/4/mlk-kpp01.stanford.edu_primarydocuments_Vol6_1948-1954MarriageCeremony.pdf
data/corpus/4/philit.stanford.edu_programs_graduateworkshop.html
data/corpus/6/transportation.stanford.edu_pdf_five-simple-checks.pdf
data/corpus/6/vaden.stanford.edu_health_library_hiv.html
data/corpus/6/vaden.stanford.edu_health_library_travel.html
data/corpus/6/virtuallabs.stanford.edu_framework_jeopardy.swf
data/corpus/6/www-group.slac.stanford.edu_esh_hazardous_activities_penetration_policies.htm
data/corpus/8/www.stanford.edu_class_humbio103_ParaSites2006_Loiasis_Clinical%2520Presentation.html
```

## Consultas conjuntivas (AND)

Siguiendo el mismo procedimiento que en la sección anterior, programa la siguiente función que calcula la intersección de dos *postings lists* dadas como argumento, las cuales han sido ordenadas por *docId*. El resultado debe ser la lista de *docIds* que contienen ambos términos.

In [ ]:
def and_postings(posting1, posting2):
    """Merge two postings lists, assuming they are sorted"""
    answer = list()
    
    ### Begin your code

    ### End your code

    return answer

Ahora, utiliza la función anterior para procesar la consulta `"galapagos" AND "eye"`, partiendo de las *postings lists* de cada uno de los términos obtenidas del índice.

In [ ]:
### Begin your code    
and_posting_list = ...

### End your code    

Muestra los nombres de los ficheros coincidentes con la consulta.

In [ ]:
 
retrieve_docnames(file_paths, and_posting_list)
 

Finalmente, comprueba mediante *grep* que el resultado es el esperado.

In [ ]:
!grep -rlw galapagos $(grep -rlw eye $corpus_dir)

Resultado:

```
data/corpus/4/news.stanford.edu_news_2000_november1_
```

# Realización de consultas utilizando el índice creado

Ahora, podemos combinar las funciones `and_postings` y `or_postings` definidas anteriormente para procesar cualquier consulta booleana, fusionando las *postings lists* obtenidas mediante el método `retrieve_docnames` del índice invertido. 


## Ejemplo 1:  `porcupine AND drosophila AND genes`.

In [ ]:
[len(inverted_index["genes"]), len(inverted_index["drosophila"]), len(inverted_index["porcupine"])]

In [ ]:
answer = and_postings(inverted_index["genes"], and_postings(inverted_index["drosophila"],inverted_index["porcupine"]))
answer

In [ ]:
retrieve_docnames(file_paths, answer)

De nuevo, comprobamos que la respuesta a la consulta es correcta mediante *grepping*:

In [ ]:
!grep -rlw "genes" $(grep -rlw "drosophila" $(grep -rlw "porcupine" $corpus_dir))

Si usamos `%timeit` para comparar los tiempos empleados en responder a la consulta mediante el índice construido y mediante *grepping*, veremos que empleando el índice invertido la latencia de respuesta es varios órdenes de magnitud inferior.

In [ ]:
%timeit and_postings(inverted_index["genes"], and_postings(inverted_index["drosophila"],inverted_index["porcupine"]))

In [ ]:
%timeit !grep -rlw "genes" $(grep -rlw "drosophila" $(grep -rlw "porcupine" $corpus_dir)) > /dev/null

## Ejemplo 2:  `spain AND (history OR medieval) AND (islamic OR mosque)`.

Realiza la consulta siguiendo el ejemplo 1. Al ser esta consulta mucho más larga, ¿qué diferencias hay en los tiempos con respecto a la consulta anterior?

In [ ]:
### Begin your code

### End your code

# Optimización de consultas

## Ejemplo 1: `stanford AND europe AND valencia`

In [ ]:
a = inverted_index["stanford"]
b = inverted_index["europe"]
c = inverted_index["valencia"]
[len(a), len(b), len(c)]

Como vemos, el término con menor frecuencia de aparición en el corpus es "valencia". Evaluando las tres posibles formas de realizar la intersección de dos de los tres términos de la consulta, veremos la longitud de las listas intermedias. También podemos medir el tiempo empleado en realizar la intersección de cada par de listas de los términos de la consulta. Esto nos ayudará a entender el rendimiento de cada uno de los planes de ejecución que vamos a considerar a continuación.

In [ ]:
len(and_postings(a,c))

In [ ]:
%timeit and_postings(a,c)

In [ ]:
len(and_postings(a,b))

In [ ]:
%timeit and_postings(a,b)

In [ ]:
len(and_postings(b,c))

In [ ]:
%timeit and_postings(b,c)

Como cabe esperar, la intersección que menos tiempo toma es la que se realiza sobre las *posting lists* de menor longitud (b y c). Por tanto, el plan de ejecución óptimo para la consulta será: `(europe AND valencia) and stanford`:

In [ ]:
and_postings(a,and_postings(b,c))

In [ ]:
# Plan óptimo: stanford AND (europe AND valencia)
%timeit and_postings(a,and_postings(b,c))

In [ ]:
%timeit and_postings(b,and_postings(a,c))

In [ ]:
%timeit and_postings(c,and_postings(a,b))

## Ejemplo 2: `stanford AND art AND murcia`

Consideremos ahora otra consulta en el que uno de los términos ("murcia") tiene una frecuencia de aparición en el corpus muy baja:

In [ ]:
a = inverted_index["stanford"]
b = inverted_index["art"]
c = inverted_index["murcia"]
[len(a), len(b), len(c)]

Completa ahora el ejemplo 2 siguiendo los pasos realizados en el ejemplo 1. Primero, evalúa el tiempo que se emplea en realizar la intersección de cada par de listas.

In [ ]:
#Begin your code

#End your code

Fíjate que `b AND c` es mucho más rápido que `a AND c` puesto que `len(b)` es mucho menor que `len(a)`. Además, el resultado intermedio `b AND c` es una lista de un único documento que además es un *docId* pequeño (2756 de casi 100000 documentos del corpus), por lo que realizar una nueva intersección con esta lista será mucho más rápido que si el *docId* fuera un valor grande.

Ahora, resuelve la consulta completa y evalúa el tiempo necesario para efectuar cada una de las combinaciones posibles para realizar esta consulta.

In [ ]:
#Begin your code

#End your code

Ahora vamos a simular la ocurrencia de cada uno de los tres términos en un hipotético nuevo documento cuyo *docId* es 100000:

In [ ]:
a_ = list(inverted_index["stanford"])
b_ = list(inverted_index["art"])
c_ = list(inverted_index["murcia"])
# Añadimos un nuevo posting ficticio a cada lista
a_.append(100000)
b_.append(100000)
c_.append(100000)
[len(a_), len(b_), len(c_)]

Resuelve ahora la consulta con los nuevos postings y compárala con el caso anterior:

In [ ]:
#Begin your code

#End your code

Como vemos, las *posting lists* ahora tienen un elemento más cada una, y al resultado de intersecar las tres se ha añadido el *docId* 100000, en comparación con la intersección de las *postings lists* genuinas según el corpus.

Si comparamos el tiempo necesario para la intersección de las listas originales frente a las nuevas listas (con la adición de la ocurrencia del *docId* 100000), observamos que hay una gran diferencia:

In [ ]:
#Obtén el tiempo requerido en realizar la nueva consulta.
#Begin your code

#End your code

Como vemos, puesto que la intersección mediante el recorrido de ambas listas no se completa hasta que se llega al fin de una de ellas, el tiempo empleado en cada caso depende en gran medida de los valores (*docIds*) en ambas listas. Así pues, tener una distancia entre *docIds* muy grande en una lista hace que, mediante el método de intersección basado en recorrido lineal, sea necesario comparar muchísimos elementos de la otra lista cuyos *docIds* están comprendidos en el rango entre dos *postings* de la primera lista. En el ejemplo dado, procesar la consulta  `stanford AND art AND murcia` toma entre 10 y 20 veces más tiempo si añadimos el *docId* ficticio 100000 a las *postings lists* de los tres términos. Esto da como resultado que la intersección de la *posting list* de "stanford" con el resultado de  `art AND murcia` deba recorrer un gran número de *postings* de "stanford" comprendidos en el rango 2756-100000, que sin la ocurrencia del *docId* 100000 no era necesario recorrer.